In [11]:
import sys
sys.path.append('f:/seismic/code/core')
sys.path.append('/seismic/code/core')
sys.path.append('/kaggle/input/my-seis-library/')
import kaggle_support as kgs
import importlib
import numpy as np
import matplotlib.pyplot as plt
import glob
import copy
import os
import seis_forward
import seis_forward2
import seis_test
import seis_prior
#plt.scatter([0,1],[1,2])
#fls.download_kaggle_dataset('jeroencottaar/byu-many-models-2/', fls.result_dir + '/many_full_res/')

In [ ]:
%%time
importlib.reload(kgs)
importlib.reload(seis_test)
importlib.reload(seis_forward2)
importlib.reload(seis_prior)
seis_test.run_all_tests(test_reference_mode=True)


prior test 44.46157497167587 88851826.703125
prior test 1.123679396601053e-13 1.4104176818463543e-05
capturing graph


In [9]:
import cupy as cp
import time
importlib.reload(seis_forward2)

d = kgs.load_all_train_data()[0]
d.load_to_memory()    
vec = d.velocity.to_vector()
d2 = kgs.load_all_train_data()[1]
d2.load_to_memory()    
vec2 = d2.velocity.to_vector()

kgs.profiling=False
seis_forward2.profiling=True
result, result_diff, result_adjoint = seis_forward2.vel_to_seis(vec, vec_adjoint = cp.zeros((5*999*70,1)))#, vec_adjoint = cp.zeros((5*999*70,1), dtype=kgs.base_type_gpu))
result2, result_diff, result_adjoint = seis_forward2.vel_to_seis(vec2, vec_adjoint = cp.zeros((5*999*70,1)))
print(kgs.rms(result - d.seismogram.to_vector()))
print(kgs.rms(result2 - d2.seismogram.to_vector()))
seis_forward2.reset_profile()
t=time.time()
NN=20
for ii in range(NN):
    result, result_diff, result_adjoint = seis_forward2.vel_to_seis(vec, vec_adjoint = cp.zeros((5*999*70,1)))#, vec_adjoint = cp.zeros((5*999*70,1), dtype=kgs.base_type_gpu))
    result2, result_diff, result_adjoint = seis_forward2.vel_to_seis(vec2, vec_adjoint = cp.zeros((5*999*70,1)))
print((time.time()-t)/(2*NN))
seis_forward2.show_profile(2*NN)

capturing graph
capturing graph
3.274694026346925e-05
3.176099982855424e-05
0.0998832643032074
init: 0.0000
prep for source loop: 0.0010
prep for time loop: 0.0017
time loop: 0.0332
extract seis: 0.0013
prep for time loop adjoint: 0.0012
time loop adjoint: 0.0581
end adjoint: 0.0017
finish: 0.0017
